# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this sql statement to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;
```

In [1]:
# Your code here
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('lego.db')
c = conn.cursor()
c.execute('''
            SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
          ''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(df.shape)
df.head()

(4, 1)


,name
0,product_details
1,product_info
2,product_pricing
3,product_reviews


In [3]:
df['name']

0    product_details
1       product_info
2    product_pricing
3    product_reviews
Name: name, dtype: object

In [5]:
tables = df['name']
queries = []
for i in range(len(tables)):
    query = 'SELECT * FROM {}'.format(tables[i])
    queries.append(query)

d = {}
for indx, table in enumerate(tables):
    d[table] = pd.read_sql(queries[indx], conn)
    print('TABLE: {}, {} obs'.format(table, len(d[table])))
    display(d[table].head(2)) # number of rows to show for each table
    print('--------------------------------------------------------------------------------')


TABLE: product_details, 744 obs


,prod_id,prod_desc,prod_long_desc,theme_name
0,630,blah,blahblah,Classic
1,2304,blah,blahblah,DUPLO®


--------------------------------------------------------------------------------
TABLE: product_info, 744 obs


,prod_id,ages,piece_count,set_name
0,630,4+,1,Brick Separator
1,2304,1½-5,1,LEGO® DUPLO® Green Baseplate


--------------------------------------------------------------------------------
TABLE: product_pricing, 10870 obs


,prod_id,country,list_price
0,75823,US,29.99
1,75822,US,19.99


--------------------------------------------------------------------------------
TABLE: product_reviews, 744 obs


,prod_id,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,630,180.0,4.0,Very Easy,4.8,4.6
1,2304,15.0,4.4,Easy,4.0,3.3


--------------------------------------------------------------------------------


In [12]:
# OR
#Preview the tables:
for item in tables:
    table = item
    length = c.execute("""SELECT count(*) from {};""".format(table)).fetchall()
    results = c.execute("""SELECT * from {} limit 5;""".format(table)).fetchall()
    df = pd.DataFrame(results)
    df.columns = [x[0] for x in c.description]
    print(table, length, '\n', df, '\n\n')

product_details [(744,)] 
    prod_id prod_desc prod_long_desc theme_name
0      630      blah       blahblah    Classic
1     2304      blah       blahblah     DUPLO®
2     7280      blah       blahblah       City
3     7281      blah       blahblah       City
4     7499      blah       blahblah       City 


product_info [(744,)] 
    prod_id  ages  piece_count                         set_name
0      630    4+            1                  Brick Separator
1     2304  1½-5            1     LEGO® DUPLO® Green Baseplate
2     7280  5-12            2      Straight & Crossroad Plates
3     7281  5-12            2  T-Junction & Curved Road Plates
4     7499  5-12           24     Flexible and Straight Tracks 


product_pricing [(10870,)] 
    prod_id country list_price
0    75823      US      29.99
1    75822      US      19.99
2    75821      US      12.99
3    21030      US      99.99
4    21035      US      79.99 


product_reviews [(744,)] 
    prod_id  num_reviews  play_star_rating re

In [15]:
query = c.execute(''' 
                    SELECT * 
                     FROM product_pricing
                     LEFT JOIN product_details
                     USING (prod_id)
                     LEFT JOIN product_info
                     USING (prod_id)
                     LEFT JOIN product_reviews
                     USING (prod_id);
                     ''').fetchall()
prod = pd.DataFrame(query)
prod.columns = [x[0] for x in c.description]
prod.head()

,prod_id,country,list_price,prod_desc,prod_long_desc,theme_name,ages,piece_count,set_name,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,US,29.99,blah,blahblah,Angry Birds™,6-12,277,Bird Island Egg Heist,2.0,4.0,Average,4.5,4.0
1,75822,US,19.99,blah,blahblah,Angry Birds™,6-12,168,Piggy Plane Attack,2.0,4.0,Easy,5.0,4.0
2,75821,US,12.99,blah,blahblah,Angry Birds™,6-12,74,Piggy Car Escape,11.0,4.3,Easy,4.3,4.1
3,21030,US,99.99,blah,blahblah,Architecture,12+,1032,United States Capitol Building,23.0,3.6,Average,4.6,4.3
4,21035,US,79.99,blah,blahblah,Architecture,12+,744,Solomon R. Guggenheim Museum®,14.0,3.2,Challenging,4.6,4.1


In [16]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
country              10870 non-null object
list_price           10870 non-null object
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


In [17]:
prod.to_csv('lego.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity!